In [1]:
import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
import os
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 64
num_classes = 10
epochs = 100

In [3]:
train=np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_trainpicture.npy")
#train_label=np.load("/mnt/Doc/reserch1/train_type.npy")
test=np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_testpicture.npy")
val = np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_validationpicture.npy")
#test_label=np.load("/mnt/Doc/reserch1/test_type.npy")

x_train = train.astype('float32')
x_test = test.astype('float32')
x_val = val.astype('float32')
x_train /= 255
x_test /= 255
x_val /=255
del train
del test
del val

In [4]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_val.shape[0], 'val samples')


x_train shape: (9000, 128, 128, 3)
9000 train samples
1500 test samples
1500 val samples


In [5]:
# divide x_test into validation and test
#x_val = x_test[:900]
#x_test = x_test[900:]
#x_val_label = test_label[:900]
#x_test_label = test_label[900:]

In [6]:
#train_label_1=np_utils.to_categorical(train_label)
#test_label_1=np_utils.to_categorical(x_test_label)
#val_label_1=np_utils.to_categorical(x_val_label)


In [7]:
#print("validation data: {0} \ntest data: {1}".format(x_val.shape, x_test.shape))

In [8]:
input_img = Input(shape=(128, 128, 3))
x = Conv2D(16, (7, 7), padding='same')(input_img)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(64, (3, 3), padding='same')(encoded)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (7, 7), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
decoded = Activation('relu')(x)

In [9]:
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      2368      
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        12832     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
__________

In [10]:
modelCheckpoint = ModelCheckpoint(filepath ="/mnt/Doc/512pix_to_128pix/best_weight_0220.h5",
                                 monitor="val_loss",
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode="min",
                                 period=1)

In [11]:
history = model.fit(x_train, x_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, x_val),
                    callbacks=[modelCheckpoint],
                    shuffle=True)

Train on 9000 samples, validate on 1500 samples
Epoch 1/100
9000/9000 [==============================] - 44s 5ms/step - loss: 0.0591 - val_loss: 0.0258

Epoch 00001: val_loss improved from inf to 0.02583, saving model to /mnt/Doc/512pix_to_128pix/best_weight_0220.h5
Epoch 2/100
9000/9000 [==============================] - 38s 4ms/step - loss: 0.0212 - val_loss: 0.0190

Epoch 00002: val_loss improved from 0.02583 to 0.01898, saving model to /mnt/Doc/512pix_to_128pix/best_weight_0220.h5
Epoch 3/100
9000/9000 [==============================] - 38s 4ms/step - loss: 0.0194 - val_loss: 0.0174

Epoch 00003: val_loss improved from 0.01898 to 0.01736, saving model to /mnt/Doc/512pix_to_128pix/best_weight_0220.h5
Epoch 4/100
9000/9000 [==============================] - 38s 4ms/step - loss: 0.0174 - val_loss: 0.0164

Epoch 00004: val_loss improved from 0.01736 to 0.01642, saving model to /mnt/Doc/512pix_to_128pix/best_weight_0220.h5
Epoch 5/100
9000/9000 [==============================] - 38s 4ms

9000/9000 [==============================] - 38s 4ms/step - loss: 0.0105 - val_loss: 0.0101

Epoch 00043: val_loss improved from 0.01014 to 0.01011, saving model to /mnt/Doc/512pix_to_128pix/best_weight_0220.h5
Epoch 44/100
9000/9000 [==============================] - 38s 4ms/step - loss: 0.0103 - val_loss: 0.0100

Epoch 00044: val_loss improved from 0.01011 to 0.01003, saving model to /mnt/Doc/512pix_to_128pix/best_weight_0220.h5
Epoch 45/100
9000/9000 [==============================] - 38s 4ms/step - loss: 0.0103 - val_loss: 0.0099

Epoch 00045: val_loss improved from 0.01003 to 0.00995, saving model to /mnt/Doc/512pix_to_128pix/best_weight_0220.h5
Epoch 46/100
9000/9000 [==============================] - 38s 4ms/step - loss: 0.0102 - val_loss: 0.0101

Epoch 00046: val_loss did not improve from 0.00995
Epoch 47/100
9000/9000 [==============================] - 38s 4ms/step - loss: 0.0102 - val_loss: 0.0098

Epoch 00047: val_loss improved from 0.00995 to 0.00980, saving model to /mnt/D

KeyboardInterrupt: 

In [ ]:
from datetime import datetime
x = datetime.now().strftime("%Y.%m.%d %H:%M:%S")
y = datetime.now().strftime("%Y.%m.%d")
new_file_pass = os.path.join("/mnt/Doc/Autoencoder_data",y)
if os.path.exists(new_file_pass)==False:
    os.mkdir(new_file_pass)
else:
    pass
y1 = os.path.join(new_file_pass,"weight"+x+".h5")
z1 = os.path.join(new_file_pass,"model"+x+".json")
model.save_weights(y)
open(z1,"w").write(model.to_json())

In [ ]:
open("autoencodermodel1.json","w").write(model.to_json())

In [ ]:
decoded_imgs=model.predict(x_test)

In [ ]:
import matplotlib.pyplot as plt
import random

In [ ]:
decoded_imgs.astype('uint8')
plt.imshow(decoded_imgs[2])

In [ ]:
plt.imshow(x_test[2])

In [ ]:
n=10
plt.figure(figsize=(20,4))
list1=range(len(x_test))
randam1=random.sample(list1,10)
for i in range(n):
    #オリジナル画像
    picture_num=randam1[i]
    ax=plt.subplot(2,n,i+1)
    plt.imshow(x_test[picture_num])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    #変換後画像
    ax=plt.subplot(2,n,i+1+n)
    plt.imshow(decoded_imgs[picture_num])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    

In [ ]:
model.summary()

In [ ]:
input_img = Input(shape=(128, 128, 3))
x = Conv2D(32, (7, 7), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(encoded)
x = Dense(128)(x)
x = Activation("relu")(x)
x = Dense(3)(x)
out = Activation("softmax")

In [ ]:
model = Model(input_img, out)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

In [ ]:
import os 
import h5py
from keras import backend as K


In [ ]:

weights_path = ''

assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
weight_value_tuples = []
for k, name in enumerate(layer_names):
    if k >= len(model.layers):
        # 全結合層の重みは読み込まない
        break
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    if len(weight_names):
        weight_values = [g[weight_name] for weight_name in weight_names]
        layer = model.layers[k]
        symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
        if len(weight_values) != len(symbolic_weights):
            raise Exception('Layer #' + str(k) +
                            ' (named "' + layer.name +
                            '" in the current model) was found to '
                            'correspond to layer ' + name +
                            ' in the save file. '
                            'However the new layer ' + layer.name +
                            ' expects ' + str(len(symbolic_weights)) +
                            ' weights, but the saved weights have ' +
                            str(len(weight_values)) +
                            ' elements.')
        weight_value_tuples += zip(symbolic_weights, weight_values)
K.batch_set_value(weight_value_tuples)
f.close()
print('Model loaded.')

In [ ]:
layer_names

In [ ]:
g = f["conv2d_15"]
weight_names = [n.decode("utf8") for n in g.attrs["weight_names"]]

In [ ]:
#メモリ使用量確認
def print_varsize():
    import types
    print("{}{: >15}{}{: >10}{}".format('|','Variable Name','|','  Size','|'))
    print(" -------------------------- ")
    for k, v in globals().items():
        if hasattr(v, 'size') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(v.size),'|'))
        elif hasattr(v, '__len__') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(len(v)),'|'))

In [ ]:
print_varsize()

In [ ]:
del test_label_1

In [ ]:
x_val